In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ..

In [ ]:
import os

# import geopandas as gpd
import matplotlib.pyplot as plt
import pandas as pd
import seaborn.objects as so


# Create a list of the New England state FIPS codes
# ne_fips = ['09', '23', '25', '33', '44', '50']

In [ ]:
# List of New England state FIPS codes
state_abbrv_to_FIPS = {
    "CT": "09",
    "ME": "23",
    "MA": "25",
    "NH": "33",
    "RI": "44",
    "VT": "50",
    "NY": "36"
}

NE_state_abbrv_to_state = {
    'CT': 'Connecticut',
    'ME': 'Maine',
    'MA': 'Massachusetts',
    'NH': 'New Hampshire',
    'RI': 'Rhode Island',
    'VT': 'Vermont',
    'NY': 'New York'
}

NE_state_to_state_abbrv = {v: k for k, v in NE_state_abbrv_to_state.items()}

equity_data_dir = 'equitydatasets'
NE_outcomes_dir = 'NewEnglandSpecificOutcomes'

# data sets
population = pd.read_csv(os.path.join(equity_data_dir, "PopulationEstimates.csv"))
poverty = pd.read_csv(os.path.join(equity_data_dir, "Poverty2023.csv"))
unemployment = pd.read_csv(os.path.join(equity_data_dir, "UnemploymentAndIncome2023.csv"))

# cobra outcomes
highhealth = pd.read_csv(
    os.path.join(NE_outcomes_dir, "Fall116highhealthGeneratorOutcome.csv"),
    skiprows=(1,3110))
lowhealth = pd.read_csv(os.path.join(NE_outcomes_dir, "Fall116lowhealthGeneratorOutcome.csv"))
carbon = pd.read_csv(os.path.join(NE_outcomes_dir, "Fall116CarbonEmissionsOutcome.csv"))
mixedhigh = pd.read_csv(os.path.join(NE_outcomes_dir, "Fall116mixhighGeneratorOutcome.csv"))
mixedlow = pd.read_csv(os.path.join(NE_outcomes_dir, "Fall116mixlowGeneratorOutcome.csv"))
original = pd.read_csv(os.path.join(NE_outcomes_dir, "Fall116OriginalCostGeneratorOutcome.csv"))

In [ ]:
highhealth

## High Health (Total Health Benefits - high estimate)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(highhealth.dtypes)

In [ ]:
NE_states = [
    'Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont', 'New York'
]
NE_highhealth = highhealth.loc[
    highhealth['State'].isin(NE_state_abbrv_to_state.values()),
    ['State', 'County', '$ Total Health Benefits(high estimate)',
     '$ Total Health Benefits(low estimate)']
]
NE_highhealth = NE_highhealth.rename(columns={
    '$ Total Health Benefits(high estimate)': 'Health Benefits (high)',
    '$ Total Health Benefits(low estimate)': 'Health Benefits (low)',
})

NE_highhealth['State'] = NE_highhealth['State'].map(lambda x: NE_state_to_state_abbrv[x])

NE_highhealth = NE_highhealth.set_index(['State', 'County']).sort_index()

In [ ]:
NE_highhealth

# Population

In [ ]:
NE_population_mask = population['State'].isin(NE_state_abbrv_to_state.keys())

NE_population = population.loc[
    NE_population_mask & (population['Attribute'] == 'POP_ESTIMATE_2021'),
    ['State', 'Area_Name', 'Value']
]

NE_population = NE_population.rename(columns={'Area_Name': 'County', 'Value': 'Population'})

NE_population = NE_population.loc[NE_population['County'].str.contains('County')]
NE_population['County'] = NE_population['County'].str.replace(' County', '', regex=False)

assert (NE_population['Population'] % 1.).max() == 0.
NE_population['Population'] = NE_population['Population'].astype(int)
NE_population = NE_population.set_index(['State', 'County']).sort_index()['Population']

# Income

In [ ]:
NE_income_mask = unemployment['State'].isin(NE_state_abbrv_to_state.keys())

NE_income = unemployment.loc[
    NE_income_mask & (unemployment['Attribute'] == 'Median_Household_Income_2022'),
    ['State', 'Area_Name', 'Value']
]

NE_income = NE_income.rename(columns={'Area_Name': 'County', 'Value': 'Income'})

NE_income = NE_income.loc[NE_income['County'].str.contains('County')]
NE_income['County'] = NE_income['County'].str.replace(' County, ..', '', regex=True)

assert (NE_income['Income'] % 1.).max() == 0.
NE_income['Income'] = NE_income['Income'].astype(int)
NE_income = NE_income.set_index(['State', 'County']).sort_index()['Income']

In [ ]:
NE_income

# Join

In [ ]:
# TODO: use outer first, then check for NaNs
# Fix earlier code until there's no NaNs
# Currently, Connecticut has NaNs in population and income, because it only has 2020 population listed, not 2021

outer = NE_highhealth.join([NE_population, NE_income], how='outer')
outer[outer.isna().any(axis=1)]

In [ ]:
inner = NE_highhealth.join([NE_population, NE_income], how='inner').reset_index()
inner

In [ ]:
(
    so.Plot(
        inner,
        x='Income',
        y='Health Benefits (high)',
        # color='State',
        pointsize='Population')
    .add(so.Dot())
    .facet(col='State')
    .layout(size=(12, 4))
)